In [1]:
from tqdm import tqdm
import pandas as pd
import re
import emoji
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoTokenizer, AutoModel
from soynlp.normalizer import repeat_normalize
import numpy as np
from sklearn.cluster import KMeans

c:\python-capstone\review_tag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# Step 1: 데이터 로드
file_path = "C:/Users/cho03/Desktop/대학/4학년/2학기/캡스톤디자인-여행사이트/data/lodging_reviews.csv"
data = pd.read_csv(file_path)

# Step 2: 'review_text' 또는 'rating' 컬럼에 NaN이 있는 행 삭제
data = data.dropna(subset=["review_text", "rating"])
# 전처리 함수 정의
pattern = re.compile(f"[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+")
url_pattern = re.compile(
    r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"
)

def clean(x):
    x = pattern.sub(" ", x)
    x = emoji.replace_emoji(x, replace="")  # emoji 삭제
    x = url_pattern.sub("", x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

# Step 3: 리뷰 텍스트 전처리
data["review_after"] = data["review_text"].apply(clean)
data
# Step 4: 리뷰의 길이를 계산하여 'length' 컬럼에 저장
data["length"] = data["review_after"].str.len()

# Step 5: 리뷰의 길이가 10 미만인 데이터 제거
data = data[data["length"] >= 10]

# Step 6: 각 숙소(lodging_id)별 전체 리뷰를 기준으로 평균 평점과 리뷰 개수 계산
lodging_summary = (
    data.groupby("lodging_id")
    .agg(total_review_count=("rating", "count"), average_rating=("rating", "mean"))
    .reset_index()
)
# 조건 1: 전체 리뷰의 평균 평점이 4 이상인 숙소
qualified_lodging_ids = lodging_summary[lodging_summary["average_rating"] >= 4]["lodging_id"]

# Step 7: 조건을 만족하는 숙소 중 평점이 4 이상인 리뷰만 필터링
good_review = data[(data["lodging_id"].isin(qualified_lodging_ids)) & (data["rating"] >= 4)]

# Step 8: 평점이 4 이상인 리뷰 개수가 5개 이상인 숙소만 선택
final_lodging_summary = good_review.groupby("lodging_id").agg(good_review_count=("rating", "count")).reset_index()
final_lodging_ids = final_lodging_summary[final_lodging_summary["good_review_count"] >= 5]["lodging_id"]

# Step 9: 최종 필터링 - 전체 조건을 만족하는 숙소의 평점 4 이상인 리뷰만 추출
filtered_data = good_review[good_review["lodging_id"].isin(final_lodging_ids)]

In [ ]:
tag_examples = {
    "가성비": [
        "가격 대비 만족도가 정말 높았어요.",
        "비용 대비 이 정도 서비스는 예상치 못했어요.",
        "이 가격에 이런 품질이라니 기대 이상입니다.",
        "가격이 저렴하면서도 필요한 편의시설이 다 갖춰져 있어 좋았습니다.",
        "가성비 면에서 다른 곳과 비교할 수 없을 정도로 좋았어요.",
        "저렴한 가격에도 불구하고 서비스가 아주 훌륭했습니다.",
        "가격에 비해 숙소의 상태와 서비스가 만족스러웠습니다.",
        "비용을 생각하면 매우 합리적인 선택이었습니다.",
        "가격이 저렴한데도 시설이 깨끗하고 관리가 잘 되어 있었습니다.",
        "이 정도 퀄리티에 이 가격이면 누구에게나 추천하고 싶어요.",
        "가격에 비해 훨씬 좋은 숙소였습니다.",
        "합리적인 가격에 만족스러운 서비스를 경험했어요.",
        "저렴한 가격이었지만, 기대 이상으로 즐거운 숙박이었어요.",
        "비용 대비 충분히 좋은 선택이라고 생각해요.",
        "경제적인 가격에 시설도 무척 만족스러웠습니다."
    ],
    "청결": [
        "객실 청결 상태가 매우 잘 유지되고 있었습니다.",
        "방이 굉장히 깔끔하고 정돈된 느낌이었어요.",
        "숙소 전체가 위생적으로 관리되어서 편안하게 머물렀습니다.",
        "화장실과 침대가 모두 매우 청결하게 유지되어 있었어요.",
        "깨끗하게 관리된 공간이라서 마음 편히 지낼 수 있었습니다.",
        "숙소가 정돈이 잘 되어 있고 위생 관리가 철저해 보였습니다.",
        "전체적인 청결 상태가 아주 훌륭했어요.",
        "방 안이 쾌적하고 먼지 하나 없는 느낌이어서 좋았어요.",
        "위생 상태가 매우 잘 유지되고 있어 안심이 되었습니다.",
        "객실이 깨끗하게 관리되어 기분이 좋았습니다.",
        "침구와 바닥 모두 청결해서 불편함이 없었어요.",
        "위생적으로 관리된 숙소라서 마음이 편안했습니다.",
        "청결도가 높아 누구에게나 추천하고 싶은 숙소예요.",
        "정리 정돈이 잘 되어 있어 아주 쾌적했습니다.",
        "위생 관리가 철저히 이루어져서 신뢰가 갔습니다."
    ],
    "직원 만족": [
        "직원들이 정말 친절해서 기분이 좋았습니다.",
        "사장님께서 직접 챙겨주셔서 감사했습니다.",
        "서비스가 아주 세심하고 친절했어요.",
        "프론트 직원들이 친절하게 응대해 주셨어요.",
        "직원들과 사장님 모두 친절하고 따뜻했습니다.",
        "요청 사항에 대해 직원들이 빠르게 응대해 주셨어요.",
        "사장님이 친절하게 맞아주셔서 마음이 편안했어요.",
        "직원들과 사장님 모두 서비스가 훌륭합니다.",
        "친절한 직원들과 사장님 덕분에 편안하게 머물렀습니다.",
        "직원들과 사장님이 정말 배려심 깊게 대해 주셨어요.",
        "사장님이 따뜻하게 대해주셔서 인상적이었어요.",
        "직원들이 협조적이고 사장님도 친절해서 좋았습니다.",
        "서비스가 뛰어나고 직원들이 상냥했어요.",
        "사장님과 직원들 모두 친절하게 맞아주셔서 기분 좋았습니다.",
        "요청에 빠르게 대응해 주셔서 감사했어요."
    ],
     "위치": [
        "위치가 좋아서 근처 관광지로 이동하기 편리했어요.",
        "도심 한가운데 위치해 있어서 접근성이 뛰어났습니다.",
        "주변에 식당과 카페가 많아 편리했습니다.",
        "대중교통과 가까운 위치라 이동이 매우 편리했어요.",
        "관광 명소들과 가까워서 시간을 절약할 수 있었습니다.",
        "주요 시설과의 접근성이 매우 좋았어요.",
        "숙소의 위치가 아주 좋고 주변에 필요한 것이 모두 있었습니다.",
        "위치가 좋아 어디든 쉽게 갈 수 있었습니다.",
        "주변에 다양한 볼거리와 즐길 거리가 있어 만족스러웠어요.",
        "숙소가 조용한 곳에 위치해 있어서 편히 쉴 수 있었습니다.",
        "대중교통을 이용하기에 최적의 위치에 있습니다.",
        "도심과 가까워 모든 것이 쉽게 접근 가능했습니다.",
        "주요 관광지와 가까운 위치 덕분에 여행이 편리했어요.",
        "교통이 편리하고 중심지와 가까워 편리했습니다.",
        "편리한 위치 덕분에 모든 계획이 순조로웠어요."
    ],
    "가족 여행": [
        "아이들과 함께 머물기에 좋은 숙소예요.",
        "가족 단위로 오기 딱 좋아요.",
        "가족과 함께 머물기에 필요한 시설이 잘 갖춰져 있어요.",
        "부모님을 모시고 오기 좋은 숙소입니다.",
        "아이들과 부모님 모두 만족할 만한 숙소였어요.",
        "부모님과 함께 이용하기 편리했습니다.",
        "가족이 함께 즐길 수 있는 숙소입니다.",
        "아이들과 부모님이 편안하게 지낼 수 있었어요.",
        "부모님과의 가족 여행에 딱 맞는 숙소입니다.",
        "아이들이 좋아할 편의시설도 잘 갖춰져 있어요.",
        "부모님과 아이들 모두 만족한 여행이었어요.",
        "가족끼리 머물기에 완벽한 공간입니다.",
        "가족 여행에 필요한 모든 것을 갖춘 숙소입니다.",
        "부모님과 아이들 모두 즐길 수 있는 숙소예요.",
        "가족이 모두 함께 편안하게 머물 수 있었습니다."
    ],
    "연인": [
        "로맨틱한 분위기로 커플 여행에 좋습니다.",
        "남자친구와 함께 오기 딱 좋아요.",
        "특별한 날을 위해 여자친구와 방문했어요.",
        "아늑한 분위기라 남자친구와 추천합니다.",
        "연인과 오붓한 시간을 보내기 좋은 숙소입니다.",
        "여자친구와 로맨틱한 시간을 보냈어요.",
        "연인과 함께하는 특별한 장소로 추천합니다.",
        "남자친구와 함께할 아늑한 분위기입니다.",
        "기념일에 여자친구와 방문하기 딱 좋아요.",
        "남자친구와 특별한 날을 보내기 좋았어요.",
        "연인과 오붓하게 머물기 좋은 장소입니다.",
        "여자친구와 머물기에 완벽한 숙소였습니다.",
        "로맨틱한 분위기가 남자친구와 잘 맞았어요.",
        "연인과 함께하기 좋은 로맨틱한 장소입니다.",
        "커플에게 추천하는 멋진 숙소예요."
    ],
    "풍경": [
        "자연 경관이 아름다워서 힐링되는 느낌이었습니다.",
        "숙소에서 바라보는 뷰가 정말 멋졌어요.",
        "산과 강의 경치가 인상적이고 마음이 편안해졌습니다.",
        "탁 트인 전망 덕분에 기분이 상쾌해졌어요.",
        "자연 속에서 여유를 만끽할 수 있는 숙소였어요.",
        "뷰가 너무 아름다워서 계속 바라보고 싶었습니다.",
        "숙소에서 보이는 자연 경관이 너무 좋아서 기억에 남아요.",
        "풍경이 멋져서 정말 힐링이 되었습니다.",
        "아름다운 자연과 함께하는 느낌이 들어 좋았어요.",
        "숙소에서 보는 전망이 정말 일품이었습니다.",
        "자연의 아름다움을 느끼며 여유로운 시간을 보낼 수 있었습니다.",
        "탁 트인 뷰 덕분에 아침마다 기분이 상쾌했어요.",
        "숙소 주변의 자연 경관이 너무 아름다워서 힐링이 되었습니다.",
        "창밖으로 보이는 풍경이 정말 감동적이었어요.",
        "자연 속에서 조용히 쉴 수 있는 멋진 숙소였습니다."
    ]
}

# 가중치를 부여할 단어 리스트 정의
tag_keywords = {
    "가성비": ["가격", "저렴", "만족", "가성비", "비용"],
    "청결": ["청결", "깨끗", "위생", "관리", "정돈"],
    "직원 만족": ["친절", "서비스", "직원", "사장님"],
    "위치": ["위치", "접근성", "근처", "교통"],
    "가족 여행": ["가족", "아이", "부모님"],
    "연인": ["연인", "커플", "남자친구", "여자친구", "기념일"],
    "풍경": ["경관", "뷰", "자연", "전망", "풍경"]
}

In [6]:
# KCBERT 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-large")
model = AutoModel.from_pretrained("beomi/kcbert-large").to(device)

In [7]:
# 문장 임베딩 함수 정의
def get_kcbert_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().detach().numpy()
    return embedding

In [8]:
def weighted_cosine_similarity(review_embedding, tag_embedding, keywords):
    # Calculate cosine similarity between the review embedding and each tag embedding
    similarities = cosine_similarity([review_embedding], tag_embedding)[0]

    # Generate weights based on whether any of the keywords match the tag; ensure weights have the same length as similarities
    weights = [3 if any(keyword in tag_embedding for keyword in keywords) else 1 for _ in range(len(similarities))]

    # Compute weighted similarity by averaging with specified weights
    weighted_similarity = np.average(similarities, weights=weights)  # Shapes should now match
    return weighted_similarity

In [ ]:
# 문장 임베딩 생성 (진행 상황 표시)
review_embeddings = np.array([get_kcbert_embedding(text) for text in tqdm(filtered_data["review_after"], desc="Generating sentence embeddings")])

Generating sentence embeddings:   4%|▍         | 358/8792 [03:26<1:24:13,  1.67it/s]

In [ ]:
# Step 10: 예시 문장 태그 임베딩 생성
tag_embeddings = {}
for tag, sentences in tag_examples.items():
    embeddings = np.vstack([get_kcbert_embedding(sentence) for sentence in sentences])
    tag_embeddings[tag] = embeddings

In [ ]:
# KMeans 클러스터링 수행 (클러스터 수 7개로 설정)
kmeans = KMeans(n_clusters=7, random_state=42)
cluster_labels = kmeans.fit_predict(review_embeddings)

In [ ]:
# 각 리뷰의 클러스터 중심과의 유사도 계산 및 다중 태그 할당
multi_tags_per_review = []

# 유사도 임계값 설정
similarity_threshold = 0.65  # 유사도가 이 값 이상이어야 태그로 할당

In [ ]:
# 여러 임계값에 대해 유사도 계산 및 태그 할당
thresholds = [0.48, 0.49, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75]
average_similarities_per_threshold = {}

for threshold in thresholds:
    assigned_tags_per_review = []
    
    # 리뷰별 태그 할당
    for review_embedding in tqdm(review_embeddings, desc=f"리뷰별 태그 할당 중 (임계값 {threshold})"):
        tag_scores = {}

        # 각 태그와의 가중치 부여된 유사도 계산
        for tag, tag_embedding in tag_embeddings.items():
            # 가중치를 부여한 코사인 유사도 계산
            similarity = weighted_cosine_similarity(review_embedding, tag_embedding, tag_keywords[tag])
            tag_scores[tag] = similarity

        # 임계값 이상인 태그만 선택하고 상위 2~3개 태그를 할당
        review_tags = [tag for tag, score in sorted(tag_scores.items(), key=lambda x: x[1], reverse=True) if score >= threshold]
        assigned_tags_per_review.append(review_tags)

    # 빈도 계산 및 추가
    filtered_data["assigned_tags"] = assigned_tags_per_review

    review_tag_counts = []
    for tags in tqdm(filtered_data["assigned_tags"], desc="리뷰별 태그 및 빈도 계산"):
        tag_counts = Counter(tags)
        # 각 태그의 개수를 데이터프레임에 추가
        review_tag_counts.append(
            {
                "리뷰의_가성비": tag_counts.get("가성비", 0),
                "리뷰의_청결도": tag_counts.get("청결", 0),
                "리뷰의_직원_만족": tag_counts.get("직원 만족", 0),
                "리뷰의_위치": tag_counts.get("위치", 0),
                "리뷰의_가족_여행": tag_counts.get("가족 여행", 0),
                "리뷰의_연인": tag_counts.get("연인", 0),
                "리뷰의_풍경": tag_counts.get("풍경", 0),
                # 태그 개수를 추가
                "리뷰의_태그_개수": len(tags),
            }
        )

    # 리뷰별 태그 빈도 리스트를 데이터프레임으로 변환
    review_tag_counts_df = pd.DataFrame(review_tag_counts)

    # 리뷰 데이터에 리뷰별 태그 빈도 추가
    final_review_data = pd.concat([filtered_data.reset_index(drop=True), review_tag_counts_df], axis=1)

    # 숙소별 태그 빈도 계산 및 상위 2개 태그 선정
    final_tags_per_lodging = []
    lodging_ids = final_review_data["lodging_id"].unique()

    for lodging_id in tqdm(lodging_ids, desc=f"숙소별 태그 및 빈도 계산 (임계값 {threshold})"):
        keywords_all_reviews = []
        for tags in final_review_data[final_review_data["lodging_id"] == lodging_id]["assigned_tags"]:
            keywords_all_reviews.extend(tags)
        tag_counts = Counter(keywords_all_reviews)
        top_2_tags = [tag for tag, count in tag_counts.most_common(2) if count >= 3]
        final_tags_per_lodging.append(
            {
                "lodging_id": lodging_id,
                "숙소의_가성비": tag_counts.get("가성비", 0),
                "숙소의_청결도": tag_counts.get("청결", 0),
                "숙소의_직원_만족": tag_counts.get("직원 만족", 0),
                "숙소의_위치": tag_counts.get("위치", 0),
                "숙소의_가족_여행": tag_counts.get("가족 여행", 0),
                "숙소의_연인": tag_counts.get("연인", 0),
                "숙소의_풍경": tag_counts.get("풍경", 0),
                "숙소의_top_2_tags": top_2_tags,
            }
        )

    lodging_tags_specified_df = pd.DataFrame(final_tags_per_lodging)
    final_data = pd.merge(final_review_data, lodging_tags_specified_df, on="lodging_id", how="left")

    # 결과 저장
    final_data.to_csv(f"review_tags_kcbert_large({threshold}).csv", encoding="utf-8-sig", index=False)

    # 각 태그와 리뷰 간의 유사도 평균 계산
    average_similarities = []
    for i, review_embedding in enumerate(review_embeddings):
        review_tags = assigned_tags_per_review[i]
        tag_similarities = []
        for tag in review_tags:
            tag_embedding = tag_embeddings[tag]
            similarity = weighted_cosine_similarity(review_embedding, tag_embedding, tag_keywords[tag])  # Use weighted similarity
            tag_similarities.append(similarity)
        if tag_similarities:
            average_similarities.append(np.mean(tag_similarities))

    # 임계값별 평균 유사도 저장
    average_similarities_per_threshold[threshold] = np.mean(average_similarities)
    print(f"임계값 {threshold}에서의 리뷰-태그 유사도 평균:", average_similarities_per_threshold[threshold])


In [ ]:
filtered_data["assigned_tags"] = multi_tags_per_review
